In [8]:
import numpy as np
import pandas as pd
from numpy_financial import irr

# Feasibility Analysis of a wind pumped hydroelectric storage system on the Island of Brava

# Load data
VentoCurvaPotencia = np.loadtxt('VentoCurvaPotencia.txt')
#Pd = pd.read_excel('Brava_consumo.xlsx', skiprows=2, usecols=range(8, 28))  # Assuming the data starts from row 3
# Carregue o arquivo Excel
xl = pd.ExcelFile('Brava_consumo.xlsx')
#Pd = pd.read_excel('Brava_consumo.xlsx', sheet_name='Carga2010-2037')
# Leia os dados da aba "Carga2010-2037"
Pd = pd.read_excel(xl, sheet_name='Carga2010-2037', dtype=float,nrows=8762)
# Wind Turbine Groups: [Power, Quantity, Corresponding column, Reservoir volume]
Cen = np.array([
    [150, 2, 1, 1500],
    [300, 1, 2, 1500],
    [150, 3, 1, 2250],
    [225, 2, 2, 2250],
    [225, 3, 3, 3400],
    [300, 2, 2, 3000],
    [300, 3, 2, 4500]
])

Res = np.zeros((len(Cen), 14))
Res2 = np.zeros((len(Cen), 8))

for k in range(len(Cen)):
    ano = 20
    RS = Cen[k, 3]
    ERmax = RS / 3600 * 9.8 * 110
    h = 8760
    Er = np.zeros((h, ano))
    Erj = ERmax / 2
    Pt = np.zeros((h, ano))
    Pb = np.zeros((h, ano))
    Erej = np.zeros((h, ano))
    Pge = np.zeros((h, ano))
    Pwr = np.zeros((h, ano))
    nb = 0.86
    nt = 0.86
    nw = 0.97
    Te = Cen[k, 0]
    ne = Cen[k, 1]
    PDmin = 256 * 0.5

    Pwt = ne * nw * VentoCurvaPotencia[:, int(Cen[k, 2]) - 1]

    Ptmax = np.mean(Pwt)

    for i in range(ano):
        for j in range(h):
            if Pwt[j] > Pd.iloc[j, i] * 0.35:
                Pwr[j, i] = Pd.iloc[j, i] * 0.35

                if (Pwt[j] - Pwr[j, i]) * nb < (ERmax - Erj):
                    Pb[j, i] = Pwt[j] - Pwr[j, i]
                    Erj += (Pwt[j] - Pwr[j, i]) * nb
                else:
                    Pb[j, i] = (ERmax - Erj) / nb
                    Erj = ERmax
                    Erej[j, i] = (Pwt[j] - Pwr[j, i]) - (ERmax - Erj) / nb
            else:
                Pwr[j, i] = Pwt[j]

            if (Pd.iloc[j, i] - Pwr[j, i]) <= Erj * nt:
                if (Pd.iloc[j, i] - Pwr[j, i]) <= Ptmax:
                    Pt[j, i] = Pd.iloc[j, i] - Pwr[j, i]
                    Erj -= Pt[j, i] / nt
                elif Ptmax < (Pd.iloc[j, i] - Pwr[j, i]) <= (Ptmax + PDmin):
                    Pge[j, i] = PDmin
                    Pt[j, i] = (Pd.iloc[j, i] - Pwr[j, i]) - Pge[j, i]
                    Erj -= Pt[j, i] / nt
                elif (Pd.iloc[j, i] - Pwr[j, i]) > (Ptmax + PDmin):
                    Pt[j, i] = Ptmax
                    Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i] - Pt[j, i]
                    Erj -= Ptmax / nt
            elif (Pd.iloc[j, i] - Pwr[j, i]) > Erj * nt:
                if Erj * nt > Ptmax and Erj * nt < (Ptmax + PDmin):
                    if Ptmax >= PDmin and (Pd.iloc[j, i] - Pwr[j, i]) <= Ptmax:
                        Pge[j, i] = PDmin
                        Pt[j, i] = (Pd.iloc[j, i] - Pwr[j, i]) - PDmin
                        Erj -= Pt[j, i] / nt
                    else:
                        Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i]
                elif Erj * nt > (Ptmax + PDmin):
                    Pt[j, i] = Ptmax
                    Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i] - Pt[j, i]
                    Erj -= Ptmax / nt
                else:
                    Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i]

            Er[j, i] = Erj

    # Calculating results
    pW = np.sum(Pwr) / np.sum(Pd) * 100
    pH = np.sum(Pt) / np.sum(Pd) * 100
    pER = (np.sum(Pwr + Pt) / np.sum(Pd)) * 100
    ERt = np.sum(Pwr + Pt)

    pDG = np.sum(Pge) / np.sum(Pd) * 100

    # Economic Analysis
    EeC = 56554.48
    CTe = 2114.14 * Te * ne
    CTh = 1469.38 * np.max(Pt)
    CBb = 1057.95 * np.max(Pb)
    CRes = 57.4 * RS
    CTub = 88.25 * 1.3 * 2 * 250
    Ctot = (EeC + CTe + CTh + CBb + CRes + CTub) * 1.15

    AF = np.zeros((9, ano))

    for i in range(ano):
        AF[0, i] = (np.sum(Pwr[:, i]) + np.sum(Pt[:, i])) * 0.212 / 0.84 * 1.17  # Poupanca de gasoleo
        AF[1, i] = np.sum(Pwt) * 0.024 + np.sum(Pb[:, 0]) * 0.082  # Custos de manutencao e operacao
        if i < 15:
            AF[2, i] = Ctot / 15  # Amortizacao do capital investido
        if i < 15:
            AF[3, i] = 0.7 * Ctot / 15 + 0.7 * (Ctot - 0.7 * Ctot / 15 * (i)) * 0.04  # Reembolso ao banco
        AF[4, i] = AF[0, i] - AF[1, i] - AF[2, i] - AF[3, i]  # Resultados antes dos impostos
        if AF[4, i] > 0:
            AF[5, i] = AF[4, i] * 0.75  # Resultados depois dos impostos
        else:
            AF[5, i] = AF[4, i]
        AF[6, i] = AF[5, i] + AF[2, i]  # Cash flow do projeto
        AF[7, i] = AF[6, i] / ((1 + 0.14) ** (i + 1))  # Cash flow atualizado
        if i > 0:
            AF[8, i] = AF[8, i - 1] + AF[7, i]  # Cash flow atualizado acumulado

    # Pay-back
    pback = np.nan
    for i in range(ano - 1):
        if AF[8, i] * AF[8, i + 1] <= 0:
            pback = (-AF[8, i] * (i + 2) + (i + 1) * AF[8, i + 1]) / (AF[8, i + 1] - AF[8, i])

    NPV = AF[8, -1]
    TIR = irr(np.concatenate(([0.3 * Ctot], AF[6, :])))

    LCOE = np.sum((AF[1, :] + AF[3, :]) / ((1 + 0.14) ** np.arange(1, 21))) / (np.sum(ERt / ((1 + 0.14) ** np.arange(1, 21))))
 
    #Res[k, 0] = Te * ne
    #Res[k, 1] = np.max(Pb)
    #Res[k, 2] = np.max(Pt)
   # Res[k, 3] = RS
   # Res[k, 4] = pDG
   # Res[k, 5] = pH
  #  Res[k, 6] = pW
    #Res[k, 7] = pER
   # Res[k, 8] = 0
   # Res[k, 9] = Ctot
   # Res[k, 10] = NPV
   # Res[k, 11] = pback
   # Res[k, 12] = TIR
   # Res[k, 13] = LCOE
    #Res[k, :] = [Te * ne, np.max(Pb), np.max(Pt), RS, pDG, pH, pW, pER, 0, Ctot, NPV, pback, TIR, LCOE]
    

# Print results
columns = ['Wind', 'Pump', 'Hydro', 'm3', 'pHydro', 'pWind', 'pER', 'pdiss', 'Ctot', 'NPV', 'pback', 'IRR', 'LCOE']
print(columns)
#print(Res)


C:\Users\Gabriel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


['Wind', 'Pump', 'Hydro', 'm3', 'pHydro', 'pWind', 'pER', 'pdiss', 'Ctot', 'NPV', 'pback', 'IRR', 'LCOE']
